#### Projeto realizado por Isabella Rocha de Oliveira
Professor Raul Ikeda
Engenharia da Computação - Insper
Visão Computacional Avançada - 2018.2

# Aula 09 - Fluxo Ótico

Exemplo adaptado do link: https://docs.opencv.org/3.4/d7/d8b/tutorial_py_lucas_kanade.html

Referências: 
    1. Szeilisk - Cap 8.4
    1. Artigo Bouguet - Pyramidal Implementation of the Lucas Kanade Feature Tracker


In [1]:
%reset -f
%matplotlib inline

#!pip install opencv-python

import cv2 as cv
import numpy as np
import matplotlib.pyplot as plt

import time

print(cv.__version__)

3.4.2


___

1. Realizar o download das imagens no Blackboard e verificar se o notebook consegue abri-las:

In [ ]:
# Abre uma imagem em gray scale
img = cv.imread('./Imagens/frame_0050.jpg', 0)

# Exibe a imagem
cv.imshow('image',img)
cv.waitKey(0)
cv.destroyAllWindows()

___

2. Detectar alguns pontos interessantes nela usando o método visto na aula passada.

In [ ]:
# Abre a imagem
img = cv.imread('./Imagens/frame_0050.jpg')

# Converte para Grayscale
bw = cv.cvtColor(img, cv.COLOR_BGR2GRAY)

# Parametriza a funcao do OpenCV
params = dict( maxCorners = 100,
               qualityLevel = 0.3,
               minDistance = 7,
               blockSize = 7 )

# Funcao que retorna uma lista de pontos
pts = cv.goodFeaturesToTrack(bw, mask = None, **params)

## Plota os pontos na imagem:

# Gera cores de forma aleatória
color = np.random.randint(0,255,(100,3))

# Insere uma marcação em cada ponto:
for i,pt in enumerate(pts):
        a,b = pt.ravel()
        img = cv.circle(img,(a,b),5,color[i].tolist(),-1)

# Exibe a imagem
cv.imshow('image',img)
cv.waitKey(0)
cv.destroyAllWindows()

___

3. Agora vamos realizar o tracking dos pontos usando fluxo ótico. 

  * Ver o artigo do Bouguet no Blackboard para mais detalhes.
  * Fonte das imagens: http://www.cvg.reading.ac.uk/PETS2009/a.html

In [ ]:
# Parametriza a funcao do OpenCV
dt_params = dict( maxCorners = 100,
                  qualityLevel = 0.3,
                  minDistance = 7,
                  blockSize = 7 )

# Parametriza o Lucas-Kanade
lk_params = dict( winSize  = (15,15),
                  maxLevel = 2,
                  criteria = (cv.TERM_CRITERIA_EPS | cv.TERM_CRITERIA_COUNT, 10, 0.03))

# Gera cores de forma aleatória
color = np.random.randint(0,255,(100,3))

# Detecta os pontos no primeiro frame. Será usado como base na próxima imagem.
previous = cv.imread('./Imagens/frame_0050.jpg')
previous_gray = cv.cvtColor(previous, cv.COLOR_BGR2GRAY)
p0 = cv.goodFeaturesToTrack(previous_gray, mask = None, **dt_params)

# Cria uma máscara para imprimir o rastro.
mask = np.zeros_like(previous)

# Abre o restante das imagens
for i in range(51,71):

    actual = cv.imread('./Imagens/frame_{:04d}.jpg'.format(i))
    
    actual_gray = cv.cvtColor(actual, cv.COLOR_BGR2GRAY)
    
    # Calcula o Fluxo Otico
    p1, st, err = cv.calcOpticalFlowPyrLK(previous_gray, actual_gray, p0, None, **lk_params)
    
    # Seleciona somente os melhores pontos
    good_new = p1[st==1]
    good_old = p0[st==1]
    
    # Desenha as trilhas para cada ponto em p1 e p0
    for i,(new, old) in enumerate(zip(good_new, good_old)):
        a,b = new.ravel()
        c,d = old.ravel()
        mask = cv.line(mask, (a,b),(c,d), [0,0,255], 2)
        actual = cv.circle(actual,(a,b),5,color[i].tolist(),-1)
        
    img = cv.add(actual, mask)
    
    cv.imshow('frame', img)
    cv.waitKey(0)
    
    # Atualiza a imagem anterior com a imagem atual e copia os pontos.
    previous_gray = actual_gray.copy()
    p0 = good_new.reshape(-1,1,2)
    
cv.destroyAllWindows()

## Projeto 2-1: Estabilização de imagens

**Motivação**: https://www.youtube.com/watch?v=4vt7bGEen2s

Agora que você consegue capturar o fluxo ótico entre duas imagens consecultivas, vamos utilizá-lo de modo inverso: como seria uma forma de compensar eventuais oscilações na câmera?

Projete um programa que captura as imagens da webcam e realiza a estabilização da imagem. Você notará que se utilizar o programa acima, a estabilização será parcial e falha. Por que?

Você deve construir um Jupyter Notebook que utiliza o Dense Optical Flow para corrigir o problema acima. O notebook deve conter comentários acerca da solução usada.

O programa base para uso da webcam:

```python
captura = cv2.VideoCapture(0)

# Para não deixar encavalar os frames
captura.set(cv2.CAP_PROP_BUFFERSIZE, 1)
 
while(1):
    ret, frame = captura.read()
    cv2.imshow("Video", frame)
   
    # Pressione ESC para sair do loop
    k = cv2.waitKey(30) & 0xff
    if k == 27:
        break
    time.sleep(0.5)
 
captura.release()
cv2.destroyAllWindows()
```

### Entrega: 12/Set 23:59 via GitHub.

### Rubrica do Projeto:

    I. Não entregou ou entregou apenas um rascunho.
    D. O programa faz apenas uma estabilização parcial com os programas da Aula 09.
    C. Utiliza o Dense Optical Flow de uma janela no centro da imagem.
    B. Utiliza meios para compensar as faixas pretas nos cantos da imagem com algum limite.
    A. Consegue realizar a compensação em rotação no eixo de profundidade.
    
    +1/2 Conceito para implementações que comprovadamente melhoram o desempenho da estabilização.
    -1/2 Conceito se o notebook não contiver uma explicação detalhada da solução apresentada.

### Estabilização de imagens por meio do Dense Optical Flow

Para realizar a estabilização da imagem da câmera será utilizado o Dense Optical Flow. As bibliotecas utilizadas nesse projeto são a Numpy e a OpenCV.

Com ele é possível obter um vetor de fluxo dos pixels da imagem, então se houver uma movimentação de um objeto na imagem, os pixels relativos à esse objeto terão um fluxo relacionado à movimentação que ocorreu. 

Neste projeto, esse fluxo será calculado em uma janela central da imagem, de tamanho 50x50 pixels e a estabilização da imagem completa se dará pelo reposicionamento do frame atual levando em consideração o acúmulo de fluxos extraídos da janela central desde o primeiro frame obtido pela câmera, até o anterior ao atual.

Para a implementação, à princípio foi delimitado o posicionamento da janela, que é centralizada em relação ao primeiro frame e possui tamanho 50x50 pixels. A câmera também é inicializada e os frames passam a ser exibidos.

Logo após foi realizado o corte do frame inicial na delimitação indicada anteriormente.

As variáveis que servirão para guardar os acúmulos de fluxo, tanto no eixo horizontal (somax) como no vertical (somay) são inicializadas como 0 pois a janela do primeiro frame será utilizada como referência. 

Um loop infinito é inicializado e ele apenas para de rodar caso a tecla "esc" seja pressionada, interrompendo a exibição dos frames capturados pela câmera. Dentro deste loop, o frame atual é capturado e o mesmo recorte da janela central é realizado. 

Com a função `cv.calcOpticalFlowFarneback` o frame anterior e o atual são utilizados para calcular o fluxo dos pixels. Essa função retorna uma lista de matrizes com os fluxos dos pixels já separados em relação ao eixo x e ao eixo y. Com esse valor de retorno, a média dos fluxos é obtida por meio da função `np.average`, para ambos os eixos. 

Como esse valor médio de fluxo é referente ao movimento que aconteceu entre as janelas dos frames, e o objetivo é reposicionar o frame atual para onde os objetos que se movimentaram estavam no frame anterior, o valor desse fluxo médio, em ambos os eixos, é multiplicado por -1, para inverter seu valor e assim poder ser usado para a estabilização.

Ambos os valores são somados à variável de acúmulo dos fluxos e isso garante que o valor dessa variável está sempre atualizada em relação à quanto um frame tem que ser deslocado em relação ao frame inicial, pois acontece como se fosse uma soma de vetores (no caso, os fluxos de cada frame vão sendo somados e vai acontecendo a soma de vetores) e alguns movimentos vão compensando outros.

Logo após foi criada uma matriz de translação para ser utilizada como argumento na função `cv.warpAffine` que é quem realmente fará o deslocamento do frame atual. Na matriz, as variáveis acumuladoras são utilizadas para indicar o valor do deslocamento em ambos os eixos cartesianos. Nessa função também é passado como argumento o tamanho da imagem final, que é fixado como o tamanho do frame atual antes das transformações de corte da janela. Isso garante que o deslocamento do frame atual será na imagem inteira e a imagem exibida terá o tamanho completo do frame atual sempre, mesmo que a janela central seja bem menor.

O vídeo com a imagem estabilizada é exibido e antes de concluir o loop, o "frame anterior" que será utilizado na próxima rodada do laço para calcular o fluxo é atualizado para o "frame atual" e quando o loop começa novamente, todo o processo apresentado acima dentro dele é repetido.
 

In [2]:
import numpy as np
cap = cv.VideoCapture(0)
ret, frame1 = cap.read()

# Constants for the crop size
xMin = int(frame1.shape[0]/2)-25
yMin = int(frame1.shape[1]/2)-25
xMax = int(frame1.shape[0]/2)+25
yMax = int(frame1.shape[1]/2)+25

frame1crop = frame1[yMin:yMax,xMin:xMax] # this is all there is to cropping
prvs = cv.cvtColor(frame1crop,cv.COLOR_BGR2GRAY)
somax = 0
somay = 0
while(1):
    ret, frame2 = cap.read()
    cropImg = frame2[yMin:yMax,xMin:xMax] # this is all there is to cropping

    next = cv.cvtColor(cropImg,cv.COLOR_BGR2GRAY)
    flow = cv.calcOpticalFlowFarneback(prvs,next, None, 0.5, 3, 15, 3, 5, 1.2, 0)
    avflowx = np.average(flow[:,:,0])
    avflowy = np.average(flow[:,:,1])
    flowx = avflowx * (-1.0)
    flowy = avflowy * (-1.0)
    somax += flowx
    somay += flowy
    translation_matrix = np.float32([[1,0,int(somax)],[0,1,int(somay)]])
    img_translation = cv.warpAffine(frame2, translation_matrix, (frame2.shape[1], frame2.shape[0]))
    
    cv.imshow('Imagem estabilizada',img_translation)    
    
    k = cv.waitKey(30) & 0xff
    if k == 27:
        break
    elif k == ord('s'):
        cv.imwrite('opticalfb.png',img_translation)
    prvs = next
cap.release()
cv.destroyAllWindows()

A implementação apresentada acima tem limitações que podem ser reconhecidas como possíveis futuras implementações com o objetivo de melhorar este sistema de estabilização de imagens. Algumas delas seriam:
    
    - Compensação não apenas no eixo x e y do deslocamento, mas também para rotações no eixo y (profundidade).
    - Não há um tratamento de compensação nas faixas pretas laterais que aparecem quando acontece o deslocamento, isso poderia ter sido realizado com as funções scaling ou resize, também do opencv.